In [1]:
import pandas as pd
from yelp.client import Client
import requests
from bs4 import BeautifulSoup
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import nltk
from tqdm import tqdm
import regex as re
import heapq  

%matplotlib inline

In [2]:
# API_KEY = "cv-xXEAMYyEV3UEF-AcDxBCH5TgNXveK9Gs_Sh9xITWYM6ob_6zKQYSCJKRMndCT6Flckrsj99lg_7GqSmoDDRYIr-3S4Dqe7CmfuwhSY_UAiYIQC_CP2QgpE4DTW3Yx"

# client = Client(API_KEY)


In [3]:
#extract yelp reviews into reviews list

reviews = []
url = "https://www.yelp.com/biz/barboncino-pizza-and-bar-brooklyn-2"
for i in range(1,10):
    result = requests.get(url)
    html = result.text
    soup = BeautifulSoup(html, "lxml")
    review_div = soup.find_all('div', {'class':'review-content'})
    for element in review_div:
        p = element.find('p')
        if p:
            reviews.append(p.text)
    url = "https://www.yelp.com/biz/barboncino-pizza-and-bar-brooklyn-2?start=" + str(len(reviews))

In [4]:
len(reviews)

180

In [5]:
# comment_words = ' '
# stopwords = set(STOPWORDS) 
  
# for val in reviews: 

#     val = str(val) 

#     tokens = val.split() 
      
#     for i in range(len(tokens)): 
#         tokens[i] = tokens[i].lower() 
          
#     for words in tokens: 
#         comment_words = comment_words + words + ' '
  
  
# wordcloud = WordCloud(width = 800, height = 800, 
#                 background_color ='white', 
#                 stopwords = stopwords, 
#                 min_font_size = 10).generate(comment_words) 
  
# plot                       
# plt.figure(figsize = (6, 6), facecolor = None) 
# plt.imshow(wordcloud) 
# plt.axis("off");

In [6]:
#Preprocessing (lowercase + remove punctuation)
# reviews = [i.lower() for i in reviews]
# reviews = [re.sub("[^\w\s]", '', i) for i in reviews]
# reviews = [i.replace("\xa0", '') for i in reviews]

In [28]:
# Tokenize reviews
# reviews_tk = [nltk.tokenize.SpaceTokenizer().tokenize(i) for i in reviews]

In [8]:
#Tokenize with different n-grams
# def ngrams(input, n):
#     input = input.split(' ')
#     output = []
#     for i in range(len(input)-n+1):
#         output.append(input[i:i+n])
#     return output

In [62]:
def generate_ngrams(s, n):
    # Pre-process
    s = [i.lower() for i in s]
    s = [re.sub("[^\w\s]", '', i) for i in s]
    s = [i.replace("\xa0", '') for i in s]

    # Break sentence in the token, remove empty tokens
    tokens = [nltk.tokenize.SpaceTokenizer().tokenize(i) for i in s][0]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]


#[" ".join(ngram) for ngram in ngrams]


In [63]:
generate_ngrams(reviews, 2)

['yep 5',
 '5 stars',
 'stars the',
 'the pizza',
 'pizza here',
 'here is',
 'is so',
 'so damn',
 'damn delicious',
 'delicious split',
 'split the',
 'the soppressata',
 'soppressata piccante',
 'piccante pizza',
 'pizza with',
 'with hot',
 'hot honey',
 'honey and',
 'and the',
 'the speck',
 'speck pizza',
 'pizza with',
 'with 2',
 '2 friends',
 'friends if',
 'if you',
 'you like',
 'like anything',
 'anything remotely',
 'remotely spicy',
 'spicy not',
 'not talking',
 'talking really',
 'really hot',
 'hot get',
 'get the',
 'the soppressata',
 'soppressata with',
 'with the',
 'the hot',
 'hot honey',
 'honey it',
 'it is',
 'is a',
 'a game',
 'game changer',
 'changer i',
 'i had',
 'had a',
 'a hard',
 'hard time',
 'time not',
 'not woofing',
 'woofing down',
 'down the',
 'the entire',
 'entire pie',
 'pie by',
 'by myselfservice',
 'myselfservice was',
 'was fantastic',
 'fantastic our',
 'our server',
 'server kept',
 'kept up',
 'up with',
 'with our',
 'our non',
 '

In [32]:
#Find word frequencies and store in dictionary
stopwords = nltk.corpus.stopwords.words("english")

word_frequencies = {}

for review in reviews_tk:  
    for word in review:
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

In [59]:
#divide word frequencies by max frequency
max_freq = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word] / max_freq

In [61]:
# word_frequencies

In [ ]:
#find review scores of reviews less than 50 words
review_scores = {}  
for review in reviews:  
    for word in nltk.word_tokenize(review.lower()):
        if word in word_frequencies.keys():
            if len(review.split(' ')) < 60:
                if review not in review_scores.keys():
                    review_scores[review] = word_frequencies[word]
                else:
                    review_scores[review] += word_frequencies[word]

In [ ]:
summary_reviews = heapq.nlargest(5, review_scores, key=review_scores.get)

summary = ', '.join(summary_reviews)  
print(summary)